In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


In [2]:
df=pd.read_csv('mushrooms.csv')

In [3]:
df.head(5)

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
1,e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
2,e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
3,p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
4,e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g


In [4]:
print(df.shape)

(8124, 23)


# Encode the Categorical Data into Numerical Data

In [5]:
le=LabelEncoder()

#Applies transformation on each coulmn 
ds=df.apply(le.fit_transform)



In [6]:
ds.head()

,type,cap_shape,cap_surface,cap_color,bruises,odor,gill_attachment,gill_spacing,gill_size,gill_color,...,stalk_surface_below_ring,stalk_color_above_ring,stalk_color_below_ring,veil_type,veil_color,ring_number,ring_type,spore_print_color,population,habitat
0,1,5,2,4,1,6,1,0,1,4,...,2,7,7,0,2,1,4,2,3,5
1,0,5,2,9,1,0,1,0,0,4,...,2,7,7,0,2,1,4,3,2,1
2,0,0,2,8,1,3,1,0,0,5,...,2,7,7,0,2,1,4,3,2,3
3,1,5,3,8,1,6,1,0,1,5,...,2,7,7,0,2,1,4,2,3,5
4,0,5,2,3,0,5,1,1,0,4,...,2,7,7,0,2,1,0,3,0,1


In [7]:
data=ds.values
print(data.shape)
print(type(data))
print(data[:5,:])
x_data=data[:,1:]
y_data=data[:,0]

(8124, 23)
<class 'numpy.ndarray'>
[[1 5 2 4 1 6 1 0 1 4 0 3 2 2 7 7 0 2 1 4 2 3 5]
 [0 5 2 9 1 0 1 0 0 4 0 2 2 2 7 7 0 2 1 4 3 2 1]
 [0 0 2 8 1 3 1 0 0 5 0 2 2 2 7 7 0 2 1 4 3 2 3]
 [1 5 3 8 1 6 1 0 1 5 0 3 2 2 7 7 0 2 1 4 2 3 5]
 [0 5 2 3 0 5 1 1 0 4 1 3 2 2 7 7 0 2 1 0 3 0 1]]


# break data into test train


In [8]:
x_train,x_test,y_train,y_test=train_test_split(x_data,y_data,test_size=0.2)

In [9]:
print(x_train.shape,y_train.shape)
print(x_test.shape,y_test.shape)

(6499, 22) (6499,)
(1625, 22) (1625,)


In [10]:
np.unique(y_train)

array([0, 1])

# Bulding Our Classifier!

In [12]:
a=np.array([0,0,0,1,1,1,0,1])

np.sum(a==1)

4

In [13]:
def prior_prob(y_train,label):
    total_examples=y_train.shape[0]
    class_examples=np.sum(y_train==label)

    return (class_examples)/float(total_examples)


In [16]:
y=np.array([1,1,1,0,0,1,0,0,1,1])
prior_prob(y,0)

0.4

In [17]:
def conditional_prob(x_train,y_train,feature_col,feather_val,label):
    x_filtered=x_train[y_train==label]
    numerator=np.sum(x_filtered[:,feature_col]==feather_val)
    denoimator=np.sum(y_train==label)

    return numerator/float(denoimator)

## Next Step : Compute Prosterior Prob for ecah test example and make predictions

In [21]:
def predict(x_train,y_train,xtest):
    classes=np.unique(y_train)
    n_features=x_train.shape[1]
    post_probs=[]

    #compute prosterior for each class
    for label in classes:
        likelihood=1.0
        for f in range(n_features):
            cond=conditional_prob(x_train,y_train,f,xtest[f],label)
            likelihood *=cond
        prior=prior_prob(y_train,label)

        post=likelihood*prior
        post_probs.append(post)
    pred=np.argmax(post_probs)
    return pred

In [26]:
output=predict(x_train,y_train,x_test[1])
print(output)
print(y_test[1])

0
0


In [28]:
def score(x_train,y_train,x_test,y_test):
    pred=[]
    for i in range(x_test.shape[0]):
        pred_label=predict(x_train,y_train,x_test[i])
        pred.append(pred_label)
    pred=np.array(pred)
    accuracy = np.sum(pred==y_test)/y_test.shape[0]

    return accuracy


In [29]:
print(score(x_train,y_train,x_test,y_test))

0.9956923076923077
